### Parameter setting
#### * Adjustments are required according to the trained model *

In [1]:
# model parameter
first = 10
second = 10
output = 1

# model file name
modelDir =lambda x: "./models/{}models/".format(x)
def get_model_name(cut):
    return "cut_{}_{}l1_{}l2_{}l3".format(cut, first, second, output)
simplexDir = lambda x, cut, number: "simplexes/simp{}/cut{}_{}".format(x, cut, number)

# the list of filtration. Parallel process execution is possible by distributing this list.
filList = range(1,65)

In [2]:
from keras import models;
import numpy as np
import copy
import itertools 
import pickle

### Construct simplex

In [3]:
# from scipy.sparse import csr_matrix

def get_relevance(model, outputSize = 1, input_layer=True):
    if input_layer:
        layers = model.layers
    else:
        layers = model.layers[1:]
        
    # print("Number of layers is {}".format(len(layers)))
    weights = [layer.get_weights()[0] for layer in layers]
    sizes = [len(weight) for weight in weights] + [outputSize]

    offset = 0
    size = sum(sizes)
    relevance = np.identity(size)

    # print("The shape of the matrix is {}".format(relevance.shape))

    total_params = 0
    for layer_num in range(len(sizes) - 1, 0, -1):
        old_offset = offset
        offset += sizes[layer_num]

        weight = weights[layer_num - 1]
        # print(weight.shape)
        total_params += weight.shape[0] * weight.shape[1]
        weightPlus = weight * (weight > 0)
        
        for j in range(0, sizes[layer_num]):
            normalizeFactor = 0
            for i in range(sizes[layer_num - 1]):
                normalizeFactor += weightPlus[i][j]
            for i in range(sizes[layer_num - 1]):
                x, y = i + offset, j + old_offset
                if weightPlus[i][j] != 0:
                    relevance[x][y] = weightPlus[i][j] / normalizeFactor
    # print("Total parameters number is {}".format(total_params))
    return np.array(relevance) 

In [4]:
def comb(sequence):
    result = []
    for L in range(1, len(sequence)+1):
        for subset in itertools.combinations( sequence, L):
            result.append(list(subset))
    return result

In [5]:
def getSimplex(matrix, pointSequence, threshold):
    matrixSize = len(matrix)
    
    relevance = 1.0
    result = []
    #startPointからのRelevanceを計算する
    startPoint = pointSequence[0]
    for pointNumber in pointSequence:
        relevance = relevance * matrix[startPoint][pointNumber]
        startPoint = pointNumber
    #relevanceがthreshold以上だったらここまでの経路を追加する

    if relevance >= threshold:
        for e in comb(pointSequence):
            result.append(e)
        #最後の要素からの連結要素について再帰的にチェックする
        lastPoint = pointSequence[-1]
        for i in range(matrixSize):
            if matrix[lastPoint][i] > 0 and i != lastPoint:
                tempPointSequence = copy.deepcopy(pointSequence)
                tempPointSequence.append(i)
                #再帰呼び出し
                temp = getSimplex(matrix, tempPointSequence, threshold)
                #結果をresultに追加
                for e in temp:
                    for ee in comb(e):
                        result.append(ee)
    return list(map(list, set(map(tuple,result))))

In [6]:
import os
def registerSimplexOutput(filList, series, cut, x, name=None):
    if name == None:
        model = models.load_model(modelDir(series) + get_model_name(cut) + "_{}.h5".format(x))
    else: 
        model = models.load_model(name)
    matrix = get_relevance(model, input_layer = False)
    matrixSize = len(matrix)
    r = list(reversed(np.logspace(-7, 0, base=10, num=64)))
    print("Filtration: ", end = "")
    for fil in filList:
        number = r[fil - 1]
        if name == None:
            filename =  simplexDir(series, cut, x) + "/Simplex" + str(fil)
        else:
            filename = "random_simp" + name + "/Simplex" + str(fil)
        
        print( str(fil) + ", ", end="")

        saveSimplex = []
        for startPoint in range(0, matrixSize):
            simplex = getSimplex(matrix, [startPoint], number)
            saveSimplex.extend(simplex)
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        saveFile = open(filename, 'wb')
        pickle.dump(saveSimplex, saveFile)
        saveFile.close

In [7]:
volume = 100000
# log_cuts = list(map(lambda x: round(x, 2), np.logspace(0, np.log(volume) - 2, base=2, num=20))) 

In [8]:
# for logarithmic scale (to divide the data uniformly)
# log_cuts = list(map(lambda x: round(x, 2), np.logspace(0, np.log(volume) - 2, base=2, num=20))) 

data_vol = [volume, 2500, 100, 60, 30, 15]
log_cuts = list(map(lambda x: int(volume / x), data_vol))

In [9]:
tries = 30
series = [1]

In [10]:
for s in series:
    for cut in log_cuts:
        for x in range(tries):
            registerSimplexOutput(filList, s, cut, x)

Filtration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, WARNING:tensorflow:No training configuration found in the save file, so the model was *not* compiled. Compile it manually.
Filtration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, WARNING:tensorflow:No training configuration found in the save file, so the model was *not* compiled. Compile it manually.
Filtration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 5

In [11]:
# for i in range(5):
    # registerSimplexOutput(filList, i, 1, name="./random{}".format(i))